<a href="https://colab.research.google.com/github/humayun-mhk/Internship-AI-NLP-Projects-/blob/main/Fake_vs_Real.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load dataset
df = pd.read_csv("/content/Fake vs Real.csv")  # Replace with actual file path

# Drop unnecessary columns
df = df[['title', 'text', 'label']]
df.dropna(inplace=True)  # Remove missing values

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

df['text'] = df['text'].apply(clean_text)

# Tokenization and Lemmatization
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
import nltk
nltk.download('punkt_tab')

def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

df['processed_text'] = df['text'].apply(preprocess_text)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(df['processed_text'], df['label'], test_size=0.2, random_state=42)

# Convert text into TF-IDF vectors
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train Naïve Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)
y_pred_nb = nb_model.predict(X_test_tfidf)
print("Naïve Bayes Accuracy:", accuracy_score(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))

# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_tfidf, y_train)
y_pred_rf = rf_model.predict(X_test_tfidf)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

# LSTM Model Training
max_words = 5000
max_len = 500
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_len)
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_len)

lstm_model = Sequential([
    Embedding(max_words, 128, input_length=max_len),
    SpatialDropout1D(0.2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')
])
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(X_train_seq, y_train, epochs=5, batch_size=64, validation_data=(X_test_seq, y_test))

# Evaluate LSTM Model
y_pred_lstm = (lstm_model.predict(X_test_seq) > 0.5).astype("int32")
print("LSTM Accuracy:", accuracy_score(y_test, y_pred_lstm))
print(classification_report(y_test, y_pred_lstm))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Naïve Bayes Accuracy: 0.8844345459044791
              precision    recall  f1-score   support

           0       0.86      0.93      0.89      2089
           1       0.92      0.84      0.88      1952

    accuracy                           0.88      4041
   macro avg       0.89      0.88      0.88      4041
weighted avg       0.89      0.88      0.88      4041

Random Forest Accuracy: 0.949517446176689
              precision    recall  f1-score   support

           0       0.94      0.96      0.95      2089
           1       0.96      0.94      0.95      1952

    accuracy                           0.95      4041
   macro avg       0.95      0.95      0.95      4041
weighted avg       0.95      0.95      0.95      4041



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
253/253 ━━━━━━━━━━━━━━━━━━━━ 363s 1s/step - accuracy: 0.8346 - loss: 0.4024 - val_accuracy: 0.8924 - val_loss: 0.2687
Epoch 2/5
253/253 ━━━━━━━━━━━━━━━━━━━━ 380s 1s/step - accuracy: 0.9075 - loss: 0.2287 - val_accuracy: 0.9310 - val_loss: 0.1916
Epoch 3/5
253/253 ━━━━━━━━━━━━━━━━━━━━ 399s 1s/step - accuracy: 0.9465 - loss: 0.1488 - val_accuracy: 0.9302 - val_loss: 0.1972
Epoch 4/5
253/253 ━━━━━━━━━━━━━━━━━━━━ 366s 1s/step - accuracy: 0.9573 - loss: 0.1234 - val_accuracy: 0.9238 - val_loss: 0.1995
Epoch 5/5
253/253 ━━━━━━━━━━━━━━━━━━━━ 357s 1s/step - accuracy: 0.9596 - loss: 0.1183 - val_accuracy: 0.9225 - val_loss: 0.2225
127/127 ━━━━━━━━━━━━━━━━━━━━ 28s 216ms/step
LSTM Accuracy: 0.9225439247710963
              precision    recall  f1-score   support

           0       0.93      0.92      0.92      2089
           1       0.91      0.93      0.92      1952

    accuracy                           0.92      4041
   macro avg       0.92      0.92      0.92      4041
weighted a